# Ideal Gas - *Propagation of Error*

#### ASU Online Physical Chemistry Laboratory


*Jeff Yarger, School of Molecular Sciences, Arizona State University.*

jyarger@proton.me


# Introduction to Propagation of Error

Error propagation is the process of determining the uncertainty in a derived quantity based on the uncertainties in the input quantities. Uncertainties can arise from various sources, such as measurement errors, rounding errors, or systematic biases. By propagating uncertainties through a calculation, we can estimate the overall uncertainty in the final result, providing a more accurate and reliable understanding of our data.

Mastering error propagation is essential when working with experimental data, especially in fields such as **chemistry, biochemistry**, physics, engineering, and other experimental sciences. Let's explore how to perform error propagation using Google Colab (python, and the numpy and/or sympy library).

Suppose that $x, ..., z$ are measured with uncertainties $\delta x, ..., \delta z$, and the measured values are used to compute a function $q(x, ..., z)$. If the uncertainties in $x, ..., z$ are independent and random, then the uncertainty in $q$ is

<br>
\begin{equation}
\delta q = \sqrt{ ( \frac{\partial q}{\partial x} \delta x  )^2 + ... + ( \frac{\partial q}{\partial z} \delta z  )^2 }
\end{equation}
<br>

In this equation, the term $\partial q / \partial x$ represents the partial derivative of $q$ with respect to $x$. If you need help calculating partial derivatives reference any general calculus book or [wikipedia](https://en.wikipedia.org/wiki/Partial_derivative).

For example, let's assume your goal is to calculate the number of moles of an ideal gas, given the pressure $P \pm \delta P$, volume $V \pm \delta V$, and temperature $T \pm \delta T$. You would calculate the number of moles of gas as $n = \frac{PV}{RT}$. But what about the uncertainty in the number of moles of gas?

Is $\delta n = \delta P \delta V/R \delta T$?  **No, it is not!**

Using the equation for error propagation (taking the partial derivative of all dependent variables) we have:

<br>
\begin{equation}
 \frac{\partial q}{\partial P} = \frac{V}{RT}
\end{equation}
<br>
\begin{equation}
 \frac{\partial q}{\partial V} = \frac{P}{RT}
\end{equation}
<br>
\begin{equation}
 \frac{\partial q}{\partial T} = -\frac{PV}{RT^2}
\end{equation}
<br>
<br>

so,
\begin{equation}
\delta \rho = \sqrt{( \frac{V}{RT} \delta P )^2  + ( \frac{P}{RT} \delta V )^2 + ( -\frac{PV}{RT^2} \delta T  )^2}
\end{equation}
<br>

If we assume lab [STP](https://en.wikipedia.org/wiki/Standard_temperature_and_pressure) conditions of 1 atm and 25C, with a volume that would be common, like 1 L, we can easily use the idea gas law to determine the number of moles of gas.  This is something all students that have taken introductory chemistry will probably remember doing.  What we want to do here is further consider the uncertainty given that there is inherent uncertainty in the T, P and V. So, lets take reasonable uncertainties of 0.01 atm, 0.5 C and 1 mL (estimated measured uncertainties).  This would be reported as P = (1.00 $\pm$ 0.01) atm, V = (1000.0 $\pm$ 1.0) mL, and T = (25.0 $\pm$ 0.5) $^o$C.  So, what is the calculated number of moles of gas and *importantly the correct uncertainty* that should be reported?


# Python Code for Propagation of Error

In [2]:
# Load numpy and sympy libraries
import numpy as np
import sympy as sp
# Uncomment the two lines below if you want to also load scipy.stats and matplotlib python libraries for advanced statistics and plotting.
import scipy.stats as scs
import matplotlib.pyplot as plt

## Start with a Quick Density Example

From Prof. Yarger's ["Error Analysis Review"](https://biopchem.education/wp-content/uploads/2021/08/error_analysis_review.pdf)

In [3]:
m, dm = sp.symbols('m dm')
v, dv = sp.symbols('v dv')

d = m/v
dd = sp.sqrt((sp.diff(d, m) * dm)**2 + (sp.diff(d, v) * dv)**2)

d_value = d.subs({m: 32.52, v: 35.17})
dd_value = dd.subs({m: 32.52, dm: 0.15, v: 35.17, dv: 0.52})

print("Density (g/mL):", d_value)
print("Uncertainty in density (g/mL):", dd_value)

Density (g/mL): 0.924651691782770
Uncertainty in density (g/mL): 0.0143211061118770


So, 0.925 $\pm$ 0.014 g/mL, exactly the same as shown in [Error Analysis Review](https://biopchem.education/wp-content/uploads/2021/08/error_analysis_review.pdf).  Now that we have gotten the basics of sympy using an example we knew the answer for, lets move on to propagation of error for an ideal gas.

## Ideal Gas - Propagation of Error

Lets set this up with Sympy and use example measured values of P = (1.00 $\pm$ 0.01) atm, V = (1000.0 $\pm$ 1.0) mL, and T = (25.0 $\pm$ 0.5) $^o$C, to calculate the number of moles of gas (in millimoles, mmol) and *importantly the uncertainty in the number of moles of gas*.

In [5]:
# Define R in terms of L⋅atm⋅K−1⋅mol−1 - 0.08205736608096 - Wikipedia
R = 0.08205736608096
# Sympy defining P, V, T and derivatives, dP, dV, dT.
P, dP = sp.symbols('P dP')
V, dV = sp.symbols('V dV')
T, dT = sp.symbols('T dT')
# Ideal Gas Equation - Relationship between P, V, T. - N(PVT)
N = (P*V)/(R*T)
# How N changes - dN - partial derivative of P, V, T
dN = sp.sqrt((sp.diff(N, P) * dP)**2 + (sp.diff(N, V) * dV)**2 + (sp.diff(N, T) * dT)**2)

# Use values of L, atm, and K... to match R units.
N_value = N.subs({P: 0.99, V: 100, T: 298.15})
dN_value = dN.subs({P: 0.99, dP: 0.01, V: 100, dV: 1.0, T: 298.15, dT: 1.0})
# Display values of N and change (dN_value) for specific values given in subs({})
print("Number of Moles:", N_value)
print("Uncertainty in Number of Moles", dN_value)

Number of Moles: 4.04653040783402
Uncertainty in Number of Moles 0.0590959478204444


So, 40.09 $\pm$ 0.42 mmol of ideal gas under conditions of P = (1.00 $\pm$ 0.01) atm, V = (1.0 $\pm$ 0.001) L, and T = (298.15 $\pm$ 0.50) K.

# References



1.   Andrea Bocchieri, [Error Propagation with Sympy: A Comprehensive Guide for Python Users.](https://andreabocchieri.com/2023-04-13-error-propagation-with-sympy/)
2.   Jeff Yarger, [Error Analysis Review](https://biopchem.education/wp-content/uploads/2021/08/error_analysis_review.pdf), biopchem.education.

